In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10


In [2]:
mnist_train = dset.MNIST("./",train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./",train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
   

    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

cuda:0


In [5]:
loss_arr=[]
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j %100 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3058, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3681, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1979, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2212, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1792, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1375, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0789, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1008, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1155, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0683, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0807, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0363, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1272, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0581, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1103, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0488, device='cuda:0', grad_fn=<NllLossBackward

In [6]:
correct = 0
total = 0

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _,output_index = torch.max(output,1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()
    
    print("Accuracy of Test Data : {}".format(100*correct/total))

Accuracy of Test Data : 98.51763153076172
